# Face Recognition

In [80]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Encoding image into 128-d vector

In [3]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [5]:
print("Total parameters: = ", FRmodel.count_params())

Total parameters: =  3743280


### 2. Triplet loss

In [69]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor,positive,negative = y_pred[0],y_pred[1],y_pred[2]
    
    pos_distance = tf.reduce_sum(tf.square(anchor - positive),axis=-1)
    neg_distance = tf.reduce_sum(tf.square(anchor - negative),axis=-1)
    
    basic_loss = pos_distance - neg_distance + alpha
    
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0))
    # acc to me 
    # loss = tf.maximum(tf.reduce_sum(basic_loss),0.0)
    
    return loss

In [70]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(test.run(loss)))

loss = 528.1432


### 3. Loading trained model

In [74]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

### 4. Applying the model

#### 4.1 Face varification 

In [98]:
database = {}
database["rajpal"] = img_to_encoding("images/rajpal.jpg", FRmodel)
database["jyoti"] = img_to_encoding("images/jyoti.jpg", FRmodel)

In [105]:
def verify(image_path, identity, database, model):
    encoding = img_to_encoding(image_path, model)
    
    dist = np.linalg.norm(encoding - database[identity])
    if dist < 0.65:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    return dist, door_open

In [116]:
verify("images/GCUI5107.JPG", "jyoti", database, FRmodel)

It's not jyoti, please go away


(0.8172667, False)

#### 4.2 Face recognition

In [110]:
def who_is_it(image_path, database, model):
    encoding = img_to_encoding(image_path, model)
    min_dist = 100
    
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding - db_enc)
        if dist < min_dist:
            min_dist = dist
            identity = name
            
    if min_dist > 0.65:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [119]:
who_is_it("images/jyoti-test2.jpg", database, FRmodel)

it's jyoti, the distance is 0.48817715


(0.48817715, 'jyoti')